In [4]:
import pandas as pd
from pathlib import Path
import numpy as np
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, f1_score
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, MaxPooling1D, Flatten, Dense, Dropout
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.callbacks import EarlyStopping
import joblib
import json

base_path = Path("C:/Users/muhfh/OneDrive/Documents/College/6th Semester/Monitoring_Las/ml")  # Ganti sesuai path lokalmu
acc_path = base_path / "jok_acc_1g.csv"
gyr_path = base_path / "jok_gyr_1g.csv"
mag_path = base_path / "jok_mag_1g.csv"

# 3. Load masing-masing file
acc_df = pd.read_csv(acc_path)
gyr_df = pd.read_csv(gyr_path)
mag_df = pd.read_csv(mag_path)

# 4. Tampilkan contoh isi file
acc_df.head()

,epoc (ms),timestamp (+0700),elapsed (s),x-axis (g),y-axis (g),z-axis (g),actid,subid-gradeid,positionid,dicontinuityid
0,1.610000e+12,2021-01-22T13.51.51.072,0.00,0.671,-0.671,0.793,Welding,3,1,1
1,1.610000e+12,2021-01-22T13.51.51.112,0.04,1.037,-1.402,1.280,Welding,3,1,1
2,1.610000e+12,2021-01-22T13.51.51.153,0.08,1.463,-0.732,1.402,Welding,3,1,1
3,1.610000e+12,2021-01-22T13.51.51.192,0.12,0.854,0.061,0.854,Welding,3,1,1
4,1.610000e+12,2021-01-22T13.51.51.233,0.16,1.220,-0.854,3.171,Welding,3,1,1


In [6]:
# Rename kolom axis agar tidak bentrok saat merge
acc_df = acc_df.rename(columns={
    'timestamp (+0700)': 'timestamp','x-axis (g)': 'acc_x', 'y-axis (g)': 'acc_y', 'z-axis (g)': 'acc_z'
})
gyr_df = gyr_df.rename(columns={
    'timestamp (+0700)': 'timestamp','x-axis (deg/s)': 'gyr_x', 'y-axis (deg/s)': 'gyr_y', 'z-axis (deg/s)': 'gyr_z'
})
mag_df = mag_df.rename(columns={
    'timestamp (+0700)': 'timestamp','x-axis (T)': 'mag_x', 'y-axis (T)': 'mag_y', 'z-axis (T)': 'mag_z'
})

# Gabungkan data berdasarkan kolom 'timestamp'
merged_df = acc_df[['timestamp', 'acc_x', 'acc_y', 'acc_z', 'actid']].merge(
    gyr_df[['timestamp', 'gyr_x', 'gyr_y', 'gyr_z']],
    on='timestamp'
).merge(
    mag_df[['timestamp', 'mag_x', 'mag_y', 'mag_z']],
    on='timestamp'
)

# Konversi kolom target menjadi kategori
merged_df['actid'] = merged_df['actid'].astype('category')

# Lihat ringkasan hasil
merged_df.info()
merged_df.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 65182 entries, 0 to 65181
Data columns (total 11 columns):
 #   Column     Non-Null Count  Dtype   
---  ------     --------------  -----   
 0   timestamp  65182 non-null  object  
 1   acc_x      65182 non-null  float64 
 2   acc_y      65182 non-null  float64 
 3   acc_z      65182 non-null  float64 
 4   actid      65182 non-null  category
 5   gyr_x      65182 non-null  float64 
 6   gyr_y      65182 non-null  float64 
 7   gyr_z      65182 non-null  float64 
 8   mag_x      65182 non-null  float64 
 9   mag_y      65182 non-null  float64 
 10  mag_z      65182 non-null  float64 
dtypes: category(1), float64(9), object(1)
memory usage: 5.0+ MB


,timestamp,acc_x,acc_y,acc_z,actid,gyr_x,gyr_y,gyr_z,mag_x,mag_y,mag_z
0,2021-01-22T13.51.51.072,0.671,-0.671,0.793,Welding,0.671,-0.671,0.793,0.000004,0.000010,-3.750000e-07
1,2021-01-22T13.51.51.112,1.037,-1.402,1.280,Welding,1.037,-1.402,1.280,0.000003,0.000010,3.750000e-07
2,2021-01-22T13.51.51.153,1.463,-0.732,1.402,Welding,1.463,-0.732,1.402,0.000003,0.000011,-3.750000e-07
3,2021-01-22T13.51.51.192,0.854,0.061,0.854,Welding,0.854,0.061,0.854,0.000003,0.000010,-1.125000e-06
4,2021-01-22T13.51.51.233,1.220,-0.854,3.171,Welding,1.220,-0.854,3.171,0.000004,0.000009,-3.750000e-07


In [7]:
# 1. Sliding window - bentuk time series untuk CNN input
WINDOW_SIZE = 25  # setiap prediksi melihat 25 data sebelumnya

def create_windows(data, labels, window_size):
    X, y = [], []
    for i in range(len(data) - window_size):
        X.append(data[i:i+window_size])
        y.append(labels[i+window_size])
    return np.array(X), np.array(y)

# Ambil fitur numerik dan label
X_all = merged_df[['acc_x', 'acc_y', 'acc_z', 'gyr_x', 'gyr_y', 'gyr_z', 'mag_x', 'mag_y', 'mag_z']].values
y_all = merged_df['actid'].values

# Encode label (ke angka)
label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(y_all)

# Buat windowed dataset
X_seq, y_seq = create_windows(X_all, y_encoded, WINDOW_SIZE)
print(f"Bentuk input CNN: {X_seq.shape}, Label: {y_seq.shape}")

# One-hot encoding label
y_categorical = to_categorical(y_seq)

Bentuk input CNN: (65157, 25, 9), Label: (65157,)


In [8]:
# 2. Bagi data train/test
X_train, X_test, y_train, y_test = train_test_split(
    X_seq, y_categorical, test_size=0.2, random_state=42, stratify=y_seq
)

In [9]:
# 3. Bangun model CNN 1D
model = Sequential([
    Conv1D(64, kernel_size=3, activation='relu', input_shape=(WINDOW_SIZE, X_seq.shape[2])),
    MaxPooling1D(pool_size=2),
    Dropout(0.3),
    Conv1D(128, kernel_size=3, activation='relu'),
    MaxPooling1D(pool_size=2),
    Flatten(),
    Dense(64, activation='relu'),
    Dropout(0.4),
    Dense(y_categorical.shape[1], activation='softmax')
])

model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.summary()

c:\Users\muhfh\OneDrive\Documents\College\6th Semester\Monitoring_Las\web_app\app_venv\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:113: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv1d (Conv1D)                 │ (None, 23, 64)         │         1,792 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling1d (MaxPooling1D)    │ (None, 11, 64)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 11, 64)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1d_1 (Conv1D)               │ (None, 9, 128)         │        24,704 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling1d_1 (MaxPooling1D)  │ (None, 4, 128)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 512)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 64)             │        32,832 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 64)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 5)              │           325 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 59,653 (233.02 KB)

 Trainable params: 59,653 (233.02 KB)

 Non-trainable params: 0 (0.00 B)

In [10]:
# 4. Train model
early_stop = EarlyStopping(patience=5, restore_best_weights=True)
history = model.fit(X_train, y_train, validation_split=0.2,
                    epochs=30, batch_size=64, callbacks=[early_stop])


Epoch 1/30
652/652 ━━━━━━━━━━━━━━━━━━━━ 3s 3ms/step - accuracy: 0.4904 - loss: 1.9302 - val_accuracy: 0.6740 - val_loss: 0.9231
Epoch 2/30
652/652 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - accuracy: 0.6920 - loss: 0.9304 - val_accuracy: 0.6996 - val_loss: 0.8618
Epoch 3/30
652/652 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - accuracy: 0.7047 - loss: 0.8739 - val_accuracy: 0.7036 - val_loss: 0.8116
Epoch 4/30
652/652 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - accuracy: 0.7061 - loss: 0.8397 - val_accuracy: 0.7075 - val_loss: 0.7957
Epoch 5/30
652/652 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - accuracy: 0.7095 - loss: 0.8203 - val_accuracy: 0.7121 - val_loss: 0.7739
Epoch 6/30
652/652 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - accuracy: 0.7097 - loss: 0.8081 - val_accuracy: 0.7131 - val_loss: 0.7648
Epoch 7/30
652/652 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - accuracy: 0.7128 - loss: 0.7948 - val_accuracy: 0.7119 - val_loss: 0.7574
Epoch 8/30
652/652 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - accuracy: 0.7102 - loss: 0.7919 - val_accuracy: 0.

In [11]:
# 5. Evaluasi
y_pred_proba = model.predict(X_test)
y_pred = np.argmax(y_pred_proba, axis=1)
y_true = np.argmax(y_test, axis=1)

print("Classification Report:\n")
print(classification_report(y_true, y_pred, target_names=label_encoder.classes_))
f1 = f1_score(y_true, y_pred, average='weighted')
print(f"F1-score: {f1:.4f}")

408/408 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step
Classification Report:

               precision    recall  f1-score   support

     Grinding       0.86      0.51      0.64      1321
       Others       0.74      0.94      0.83      5740
  Preparation       0.84      0.24      0.38       877
Slag Cleaning       0.84      0.54      0.66      1311
      Welding       0.96      0.98      0.97      3783

     accuracy                           0.82     13032
    macro avg       0.85      0.64      0.69     13032
 weighted avg       0.83      0.82      0.80     13032

F1-score: 0.8016


In [12]:
# 6. Simpan model dan label encoder
model_path = base_path / "cnn1d_welder_model.h5"
model.save(model_path)

# Simpan label encoder mapping
label_map_path = base_path / "label_mapping.json"
with open(label_map_path, 'w') as f:
    json.dump(dict(zip(map(str, range(len(label_encoder.classes_))), label_encoder.classes_)), f)

print(f"Model dan label mapping disimpan ke: {model_path} & {label_map_path}")

Model dan label mapping disimpan ke: C:\Users\muhfh\OneDrive\Documents\College\6th Semester\Monitoring_Las\ml\cnn1d_welder_model.h5 & C:\Users\muhfh\OneDrive\Documents\College\6th Semester\Monitoring_Las\ml\label_mapping.json
